1. Get crv emissions per pool.
2. Get fees claimed per pool.
3. divide the two to get swap revenue per crv emitted per gauge.

In [1]:
# subgraph querying modules
import datetime
import calendar

In [2]:
# plotting tools
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")

In [3]:
# brownie
import os
import sys
sys.path.append("../../")

import brownie

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect
from utils.eth_blocks_utils import get_block_for_timestamp


In [ ]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

# Pools and Gauges

Get dict of pools (keys) and gauge (value)

In [ ]:
ethereum_registry_contract = init_contract("0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5")
ethereum_registry_contract.info()

Get pools and gauges:

In [ ]:
pool_count = ethereum_registry_contract.pool_count()
eth_pools_and_gauges = {}
for pool_id in range(pool_count):

    pool_addr = ethereum_registry_contract.pool_list(pool_id)
    gauge_addr = ethereum_registry_contract.get_gauges(pool_addr)
    lp_token_addr = ethereum_registry_contract.get_lp_token(pool_addr)

    # get lp_token name:
    pool_token_contract = init_contract(lp_token_addr)
    pool_token_name = pool_token_contract.name()

    eth_pools_and_gauges[pool_addr] = [gauge_addr[0][0], lp_token_addr, pool_token_name]

eth_pools_and_gauges

# CRV emissions per pool:

In [ ]:
crv_token_addr = "0xD533a949740bb3306d119CC777fa900bA034cd52"
crv_token = init_contract(crv_token_addr)
crv_token.info()

CRV emissions rate per block

In [ ]:
crv_token.rate() * 1e-18

get number of blocks between two timestamps:

In [ ]:
block_time_start = int(
    calendar.timegm((datetime.datetime.now() - datetime.timedelta(days=7)).date().timetuple())
)
block_time_end = int(
    calendar.timegm((datetime.datetime.now()).date().timetuple())
)
crv_token.mintable_in_timeframe(block_time_start, block_time_end) * 1e-18

get gauge weights from registry for block:

In [ ]:
gauge_controller_address = "0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB"
gauge_controller = init_contract(gauge_controller_address)
gauge_controller.info()

Get crv emitted for gauge

In [ ]:
time_start = int(datetime.datetime.strptime("2021-10-24", "%Y-%m-%d").timestamp())
time_end = int(datetime.datetime.strptime("2021-10-31", "%Y-%m-%d").timestamp())
delta_time = int(datetime.timedelta(days=1).total_seconds())
date_range = list(range(time_start,time_end,delta_time))

In [ ]:
emissions_data = pd.DataFrame()
for timestamp in date_range:

    block_number = get_block_for_timestamp(timestamp=timestamp)

    with brownie.multicall(block_identifier=block_number):

        for pool_addr, pool_details in eth_pools_and_gauges.items():

            pool_data = {}

            gauge_addr = pool_details[0]
            lp_token_addr = pool_details[1]
            pool_name = pool_details[2]

            # get crv emissions for timestamp from the past day:
            crv_minted = crv_token.mintable_in_timeframe(
                int(timestamp - delta_time),
                timestamp
            ) * 1e-18

            # get gauge relative weight
            gauge_relative_weight = gauge_controller.gauge_relative_weight(gauge_addr) * 1e-18

            # get virtual price:
            try:
                lp_token_virtual_price = (
                    ethereum_registry_contract.get_virtual_price_from_lp_token(lp_token_addr)
                ) * 1e-18
            except TypeError:
                lp_token = init_contract(lp_token_addr)
                lp_token_virtual_price = lp_token.get_virtual_price() * 1e-18

            # get total supply of lp_token:
            lp_token = init_contract(lp_token_addr)
            total_supply_lp_token = lp_token.totalSupply() * 1e-18

            # pool data
            pool_data['timestamp'] = timestamp
            pool_data['block_number'] = block_number
            pool_data['pool_name'] = pool_name
            pool_data['pool_addr'] = pool_addr
            pool_data['lp_token_addr'] = lp_token_addr
            pool_data['gauge_addr'] = gauge_addr
            pool_data['gauge_relative_weight'] = gauge_relative_weight
            pool_data['lp_token_virtual_price'] = lp_token_virtual_price
            pool_data['total_supply_lp_token'] = total_supply_lp_token
            pool_data['crv_minted_past_1d'] = crv_minted
            pool_data['crv_to_gauge'] = gauge_relative_weight * crv_minted

            pool_data = pd.DataFrame(
                data=pool_data,
                index=[pd.to_datetime(pool_data['timestamp'], unit='s')]
            )

            # join df
            emissions_data = pd.concat([emissions_data, pool_data])


In [ ]:
emissions_data

# swap fees to liquidity providers per pool per week:

In [ ]:
emissions_and_revenue = pd.DataFrame()
for pool_name in emissions_data.pool_name.unique():

    pool_data = emissions_data[emissions_data.pool_name == pool_name]
    pool_data = pool_data.sort_index(ascending=True)

    pool_data['virtual_price_diff'] = pool_data.lp_token_virtual_price.diff()
    pool_data['swap_fee_revenue'] = pool_data.virtual_price_diff * pool_data.total_supply_lp_token * 2
    pool_data['swap_revenue_to_crv_emissions_ratio'] = pool_data.swap_fee_revenue / pool_data.crv_to_gauge

    emissions_and_revenue = pd.concat([emissions_and_revenue, pool_data])

In [ ]:
emissions_and_revenue = emissions_and_revenue.dropna()
emissions_and_revenue

# plots:

In [ ]:
emissions_and_revenue[emissions_and_revenue.lp_token_addr == "0xc4AD29ba4B3c580e6D59105FFf484999997675Ff"]

In [ ]:
1.003108 - 1.003055

In [ ]:
time_start = datetime.datetime.strptime('2021-10-24', '%Y-%m-%d')
time_end = datetime.datetime.strptime('2021-10-31', '%Y-%m-%d')
time_end

In [ ]:
emissions_and_revenue.loc[(emissions_and_revenue.index > time_start) & (emissions_and_revenue.index < time_end)]

In [ ]:
df_plot = emissions_and_revenue.loc[
    (emissions_and_revenue.index > time_start) & (emissions_and_revenue.index < time_end)
].groupby('pool_name').sum().sort_values(
    by='swap_fee_revenue',
    ascending=False
)

fig, ax = plt.subplots(1,1, figsize=(8, 20))

df_plot[["swap_revenue_to_crv_emissions_ratio"]].plot.barh(
    # kind='bar',
    ax=ax,
    color='red',
    width=0.4,
    position=1,
    align='center'
)
ax.set_ylabel("", fontsize=20)
ax.set_xlabel("Swap Fee Revenue / CRV emissions [ $ ]", fontsize=20, color='black')
ax.tick_params(axis='both', which='major', labelsize=20)
ax.tick_params(axis='x', which='major', colors='black')
ax.get_legend().remove()

ax.set_xlim([0, 50])

# ax2 = ax.twinx()
# (df_plot[["crv_to_gauge"]]).plot(
#     kind='bar',
#     ax=ax2,
#     color='blue',
#     width=0.4,
#     position=2,
# )
#
# ax2.get_legend().remove()
# ax2.grid(b=None)
# ax2.set_ylabel("crv for pool [ $/CRV ]", fontsize=20, color='blue')
# ax2.tick_params(axis='y', which='major', labelsize=20, colors='blue')
